In [68]:
!pip install pyspark
!pip install folium


In [69]:
# Create SparkSession
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("ManRattan")\
        .getOrCreate()

In [70]:
# Import Dependencies
!pip install nbformat
!pip install seaborn

from pyspark.sql.functions import *

import plotly.graph_objects as go
import numpy as np
import folium
import seaborn as sns
from folium import plugins
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use('seaborn')

In [71]:
# Read in csv
df_sightings = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema","true") \
    .load("/Users/gleason/Desktop/personal-repo/gt_project_4/Resources/nyc_sightings.csv")

df_sightings.printSchema()

root
 |-- Unique Key: integer (nullable = true)
 |-- Created Date: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due Date: string (nullable = true)
 |-- Resolution Action Updated Date: string (nullable = true)
 |-- Communit

In [72]:
# Group sightings by borough
df_location = df_sightings.select('Unique Key','Incident Zip','City','Address Type','Location Type','Borough')
df_borough = df_location.groupBy("Borough").count()

In [73]:
# Drop null values
df_borough = df_borough.na.drop(subset=["Borough"])
df_borough = df_borough.filter(df_borough.Borough!="Unspecified")

In [74]:
Borough = df_borough.select(collect_list('Borough')).first()[0] 
BoroughCount = df_borough.select(collect_list('count')).first()[0]

In [75]:
fig1 = go.Figure(
    data=[go.Pie(labels = Borough , values = BoroughCount)],
    layout_title_text=("Rat Sightings by Borough")
)
fig1.show()

In [11]:
# Sightings by city
# goupby and sort
city_rats = df_sightings.groupBy("City").count()
city_rats = city_rats.sort(col('count').desc())

In [12]:
City = city_rats.select(collect_list('City')).first()[0] 
city_count = city_rats.select(collect_list('count')).first()[0]

In [13]:
fig2 = go.Figure(
    data=[go.Bar(x = City , y = city_count)],
    layout_title_text=("Rat Sightings by City")
)
fig2.show()

In [14]:
# Sightings by building type
df_building = df_location.groupBy("Location Type").count()

In [15]:
building = df_building.select(collect_list('Location Type')).first()[0] 
building_type_count = df_building.select(collect_list('count')).first()[0]

In [16]:
fig3 = go.Figure(
    data=[go.Bar(x = building , y = building_type_count)],
    layout_title_text=("Rat Sightings by Building Type")
)
fig3.show()

In [17]:
# Sightings by month
df_month = df_sightings.select('Unique Key','Created Date')
df_month= df_month.withColumn("Month",df_month["Created Date"].substr(0,2))

In [18]:
# Create Dict
monthsDict = { "01" : "January", "02": "February", "03": "March", "04": "April", "05": "May", "06": "June", "07": "July",
          "08" : "August", "09": "September", "10": "October", "11": "November", "12": "December"}

In [19]:
#groupby and sort
df_month = df_month.groupBy("Month").count()
df_month = df_month.sort(df_month.Month.asc())

months = df_month.select(collect_list('Month')).first()[0]
sightings = df_month.select(collect_list('count')).first()[0]
months = [monthsDict[x] for x in months]

In [20]:
fig4 = go.Figure([go.Bar(x=months, y=sightings, marker_color='red')], 
                layout_title_text="Rat Sightings by Month")
fig4.update_xaxes(title_text="Months")
fig4.update_yaxes(title_text="No. of Rat Sightings")
fig4.show()


In [21]:
# Sightings by count per season
SeasonDict = {
    "Winter": ["December", "January", "February"], 
    "Spring": ["March", "April", "May"],
    "Summer": ["June", "July", "August"],
    "Fall": ["September", "October", "November"]
}   

In [23]:
season_count = []
seasons = []

for season in SeasonDict:
    seasons.append(season)
    count = 0
    for month in SeasonDict[season]:
        index = months.index(month)
        count += sightings[index]
    
    season_count.append(count)

In [24]:
fig5 = go.Figure([go.Bar(x=seasons, y=season_count, marker_color='Orange')],
                layout_title_text="Rats Sightings Count Per Season")
fig5.update_xaxes(title_text=("Seasons"))
fig5.update_yaxes(title_text=("No. of Rat Sightings"))
fig5.show()

In [25]:
# Sightings vs seasons
fig6 = go.Figure(
    data=[go.Pie(labels = seasons , values = season_count)],
    layout_title_text=("Rat Sightings vs Seasons")
)
fig6.show()

In [26]:
# Sightings by year
df_year = df_sightings.select('Unique Key','Created Date')
df_year = df_year.withColumn("Year",df_year["Created Date"].substr(0,10).substr(7,8))

In [27]:
# groupby and sort
df_year = df_year.groupBy("Year").count()
df_year = df_year.sort(df_year.Year.asc())
years = df_year.select(collect_list('Year')).first()[0]
sightingsCountPerYear = df_year.select(collect_list('count')).first()[0]

In [28]:
fig7 = go.Figure([go.Scatter(x=years, y=sightingsCountPerYear, marker_color='blue')],
                layout_title_text="No. of Rat Sightings vs Year")
fig7.update_xaxes(title_text="Year")
fig7.update_yaxes(title_text="No. of Rats Sightings")
fig7.show()


# NYC restaurant code violations

In [34]:
df_inspection = spark.read\
    .option("header","true")\
    .option("inferSchema","true")\
    .option("delimiter",',')\
    .csv('/Users/gleason/Desktop/personal-repo/gt_project_4/Resources/nyc_inspection.csv')

df_inspection.printSchema()

root
 |-- CAMIS: integer (nullable = true)
 |-- DBA: string (nullable = true)
 |-- BORO: string (nullable = true)
 |-- BUILDING: string (nullable = true)
 |-- STREET: string (nullable = true)
 |-- ZIPCODE: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- CUISINE DESCRIPTION: string (nullable = true)
 |-- INSPECTION DATE: string (nullable = true)
 |-- ACTION: string (nullable = true)
 |-- VIOLATION CODE: string (nullable = true)
 |-- VIOLATION DESCRIPTION: string (nullable = true)
 |-- CRITICAL FLAG: string (nullable = true)
 |-- SCORE: integer (nullable = true)
 |-- GRADE: string (nullable = true)
 |-- GRADE DATE: string (nullable = true)
 |-- RECORD DATE: string (nullable = true)
 |-- INSPECTION TYPE: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Community Board: integer (nullable = true)
 |-- Council District: integer (nullable = true)
 |-- Census Tract: integer (nullable = true)
 |-- BIN: integer (nul

In [40]:
# Group violations by borough
df_location = df_inspection.select("BORO", "VIOLATION CODE", "VIOLATION DESCRIPTION", "Latitude", "Longitude")
                           
df_borough = df_location.groupBy("BORO").count()

In [52]:
df_location = df_location.dropna(subset=['VIOLATION CODE'])
df_location.show(5)

+---------+--------------+---------------------+---------------+----------------+
|     BORO|VIOLATION CODE|VIOLATION DESCRIPTION|       Latitude|       Longitude|
+---------+--------------+---------------------+---------------+----------------+
|    Bronx|           09B| Thawing procedure...|            0.0|             0.0|
|    Bronx|           09C| Food contact surf...|            0.0|             0.0|
|Manhattan|           09B| Thawing procedure...|40.761948488597|-73.982622296463|
|   Queens|           10H| Proper sanitizati...|40.648312826068|-73.788281497594|
|    Bronx|           02B| Hot TCS food item...|            0.0|             0.0|
+---------+--------------+---------------------+---------------+----------------+
only showing top 5 rows



In [54]:
import pandas as pd
df_location = df_location.replace(0.0, pd.np.nan)
df_location = df_location.dropna()

/Users/gleason/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [58]:
df_rodents = df_location[df_location["VIOLATION CODE"] == "04L"]
df_rodents.show(10)

+-------------+--------------+---------------------+---------------+----------------+
|         BORO|VIOLATION CODE|VIOLATION DESCRIPTION|       Latitude|       Longitude|
+-------------+--------------+---------------------+---------------+----------------+
|     Brooklyn|           04L| Evidence of mice ...|40.656636664405|-73.930559688328|
|       Queens|           04L| Evidence of mice ...|40.693307869085|-73.853069565193|
|Staten Island|           04L| Evidence of mice ...|40.637171578333|-74.128693798627|
|    Manhattan|           04L| Evidence of mice ...|40.710529012969|-74.009378280217|
|     Brooklyn|           04L| Evidence of mice ...|40.663518587577| -73.98437458362|
|       Queens|           04L| Evidence of mice ...|40.762684950086|-73.912460727824|
|    Manhattan|           04L| Evidence of mice ...|40.757357634315|-73.993416132521|
|       Queens|           04L| Evidence of mice ...|40.683952391992|-73.847883782115|
|    Manhattan|           04L| Evidence of mice ...|40

In [ ]:
import folium

# create a map centered on New York City
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# loop over each row in the DataFrame and add a marker to the map
for row in df_rodents.collect():
    popup = row["VIOLATION DESCRIPTION"]
    lat, lon = row["Latitude"], row["Longitude"]
    folium.Marker(location=[lat, lon], popup=popup).add_to(nyc_map)

# display the map
nyc_map

